<a href="https://colab.research.google.com/github/stevecoffee/dasc5300/blob/main/Project2/DASC_5300_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [191]:
!pip install python-igraph

In [192]:
import pandas as pd
from igraph import *    # https://igraph.org/python/tutorial/0.9.6/tutorial.html

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

Grabbed additional data from https://openflights.org/data.html:
- airports.dat
- airports-extended.dat
- routes.dat
- countries.dat


In [193]:
# load countries from openflight data
df_openflight_countries = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/OpenFlights/countries.dat',
    names=['name', 'iso-code', 'dafif-code'],
    skipinitialspace='true', header=0, na_values='\\N')
df_openflight_countries

,name,iso-code,dafif-code
0,Aruba,AW,AA
1,Antigua and Barbuda,AG,AC
2,United Arab Emirates,AE,AE
3,Afghanistan,AF,AF
4,Algeria,DZ,AG
...,...,...,...
255,Samoa,WS,WS
256,Eswatini,SZ,WZ
257,Yemen,YE,YM
258,Zambia,ZM,ZA


In [194]:
# load airports from openflight data
df_openflight_airports = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/OpenFlights/airports.dat',
    names=['airport-id', 'name', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'timezone', 'DST', 'database-timezone', 'type', 'source'],
    skipinitialspace='true', header=0, na_values='\\N')
# df_openflight_airports.reset_index(level=0, inplace=True)  # add row number as a column to use as vertex ID with graph library
df_openflight_airports['airport-id'] = df_openflight_airports['airport-id'].astype(str)  # change from int to string for graphing lib
df_openflight_airports = df_openflight_airports[ df_openflight_airports['iata'].notna() ]
print(df_openflight_airports.count())
print(df_openflight_airports[df_openflight_airports.duplicated(['iata'], False)])
# print(df_openflight_airports[ df_openflight_airports['iata'] == 'YGR' ])


airport-id           6071
name                 6071
city                 6032
country              6071
iata                 6071
icao                 6071
latitude             6071
longitude            6071
altitude             6071
timezone             5867
DST                  5867
database-timezone    5514
type                 6071
source               6071
dtype: int64
Empty DataFrame
Columns: [airport-id, name, city, country, iata, icao, latitude, longitude, altitude, timezone, DST, database-timezone, type, source]
Index: []


In [195]:
# load airports from data file provided with project
df_provided_airports = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/provided/AirportMapping.csv',
    skipinitialspace='true', na_values='\\N')
df_provided_airports.drop_duplicates('Code', inplace=True)
df_provided_airports.dropna(inplace=True)
print(df_provided_airports.count())
print(df_provided_airports[ df_provided_airports['Code'].isna() ])
# print(df_provided_airports[ df_provided_airports['Code'] == 'YGR' ])


City       1497
Country    1497
Code       1497
dtype: int64
Empty DataFrame
Columns: [City, Country, Code]
Index: []


In [196]:
df_merged_airports = df_provided_airports.merge(df_openflight_airports, how='inner', left_on='Code', right_on='iata')
print(df_merged_airports.count())
print(df_merged_airports[df_merged_airports.duplicated(['Code'], False)])


City                 1448
Country              1448
Code                 1448
airport-id           1448
name                 1448
city                 1447
country              1448
iata                 1448
icao                 1448
latitude             1448
longitude            1448
altitude             1448
timezone             1443
DST                  1443
database-timezone    1437
type                 1448
source               1448
dtype: int64
Empty DataFrame
Columns: [City, Country, Code, airport-id, name, city, country, iata, icao, latitude, longitude, altitude, timezone, DST, database-timezone, type, source]
Index: []


In [197]:
# print(df_openflight_airports[ df_openflight_airports['iata'] == 'YGR' ])
# print(df_provided_airports[ df_provided_airports['Code'] == 'YGR' ])
# print(df_merged_airports[ df_merged_airports['iata'] == 'YGR' ])
# print(df_merged_airports[ df_merged_airports['Code'] == 'YGR' ])
print(df_merged_airports)

                City      Country Code airport-id                                      name        city         country iata  icao   latitude   longitude  altitude  timezone DST     database-timezone     type       source
0            Aalborg      Denmark  AAL        628                           Aalborg Airport     Aalborg         Denmark  AAL  EKYT  57.092759    9.849243        10       1.0   E     Europe/Copenhagen  airport  OurAirports
1           Aalesund       Norway  AES        630                           Ålesund Airport     Alesund          Norway  AES  ENAL  62.562500    6.119700        69       1.0   E           Europe/Oslo  airport  OurAirports
2             Aarhus      Denmark  AAR        607                            Aarhus Airport      Aarhus         Denmark  AAR  EKAH  56.299999   10.619000        82       1.0   E     Europe/Copenhagen  airport  OurAirports
3     Abbotsford, BC       Canada  YXX        175                        Abbotsford Airport  Abbotsford         

In [198]:
# load routes from openflight data
df_openflight_routes = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/OpenFlights/routes.dat',
    names=['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment'],
    skipinitialspace='true', header=0, na_values='\\N')
print(df_openflight_routes)
# print(df_openflight_routes[['source-airport-code', 'source-airport-id']].describe())
# print(df_openflight_routes[['dest-airport-code', 'dest-airport-id']].describe())


      airline-code  airline-id source-airport-code  source-airport-id dest-airport-code  dest-airport-id codeshare  stops equipment
0               2B       410.0                 ASF             2966.0               KZN           2990.0       NaN      0       CR2
1               2B       410.0                 ASF             2966.0               MRV           2962.0       NaN      0       CR2
2               2B       410.0                 CEK             2968.0               KZN           2990.0       NaN      0       CR2
3               2B       410.0                 CEK             2968.0               OVB           4078.0       NaN      0       CR2
4               2B       410.0                 DME             4029.0               KZN           2990.0       NaN      0       CR2
...            ...         ...                 ...                ...               ...              ...       ...    ...       ...
67657           ZL      4178.0                 WYA             6334.0       

In [199]:
# load routes from data file provided with project
df_provided_routes = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/provided/routes_v3.txt',
    names=['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment'],
    skipinitialspace='true', header=0, na_values='\\N')
df_provided_routes = df_provided_routes.drop('codeshare', axis=1)
print(df_provided_routes)

df_provided_routes_al28 = df_provided_routes[df_provided_routes['airline-code'] == 'AL28']
print(df_provided_routes_al28)

      airline-code airline-id source-airport-code  source-airport-id dest-airport-code  dest-airport-id  stops equipment
0            AL242      AL242                 ASF             2966.0               KZN           2990.0      0       CR2
1            AL242      AL242                 ASF             2966.0               MRV           2962.0      0       CR2
2            AL242      AL242                 CEK             2968.0               KZN           2990.0      0       CR2
3            AL242      AL242                 CEK             2968.0               OVB           4078.0      0       CR2
4            AL242      AL242                 DME             4029.0               KZN           2990.0      0       CR2
...            ...        ...                 ...                ...               ...              ...    ...       ...
67657        AL156      AL156                 WYA             6334.0               ADL           3341.0      0       SF3
67658        AL530      AL530   

In [200]:
join_columns = ['source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'stops', 'equipment']

df_merged_routes = df_openflight_routes.merge(df_provided_routes, how='outer', on=join_columns).set_index(join_columns)
df_merged_routes = df_merged_routes[['airline-code_y', 'airline-code_x', 'airline-id_x', 'codeshare']]
print(df_merged_routes)

df_merged_routes_al28 = df_openflight_routes.merge(df_provided_routes_al28, how='outer', on=join_columns).set_index(join_columns)
df_merged_routes_al28 = df_merged_routes_al28[['airline-code_y', 'airline-code_x', 'airline-id_x', 'codeshare']]
print(df_merged_routes_al28)


                                                                                        airline-code_y airline-code_x  airline-id_x codeshare
source-airport-code source-airport-id dest-airport-code dest-airport-id stops equipment                                                      
ASF                 2966.0            KZN               2990.0          0     CR2                AL242             2B         410.0       NaN
                                      MRV               2962.0          0     CR2                AL242             2B         410.0       NaN
CEK                 2968.0            KZN               2990.0          0     CR2                AL242             2B         410.0       NaN
                                      OVB               4078.0          0     CR2                AL242             2B         410.0       NaN
DME                 4029.0            KZN               2990.0          0     CR2                AL242             2B         410.0       NaN
...   

In [201]:
test = df_merged_routes['airline-code_y'] =='AL28'
print(df_merged_routes[test])

test = df_merged_routes_al28['airline-code_y'] =='AL28'
print(df_merged_routes_al28[test])


                                                                                        airline-code_y airline-code_x  airline-id_x codeshare
source-airport-code source-airport-id dest-airport-code dest-airport-id stops equipment                                                      
AGP                 1230.0            STR               350.0           0     320 319             AL28             4U        2548.0       NaN
                                      TXL               351.0           0     319                 AL28             4U        2548.0       NaN
ALG                 210.0             CGN               344.0           0     319                 AL28             4U        2548.0       NaN
AMS                 580.0             HAM               342.0           0     CRJ                 AL28             4U        2548.0       NaN
AOI                 4309.0            DUS               345.0           0     CRJ                 AL28             4U        2548.0       NaN
...   

In [202]:
# clean openflight routes (df_openflight_routes)

# create dictionary for patching missing airport IDs from airport codes
iataLookup = df_openflight_airports.set_index('iata')['airport-id'].to_dict()

# patch source airport id where possible
missing_source_mask = df_provided_routes['source-airport-id'].isna()
df_provided_routes.loc[missing_source_mask, 'source-airport-id'] = df_provided_routes.loc[missing_source_mask, 'source-airport-code'].map(iataLookup)

# patch dest airport id where possible
missing_dest_mask = df_provided_routes['dest-airport-id'].isna()
df_provided_routes.loc[missing_dest_mask, 'dest-airport-id'] = df_provided_routes.loc[missing_dest_mask, 'dest-airport-code'].map(iataLookup)


In [203]:
print('\nStarting Data\n', df_openflight_routes)

#remove unneeded columns
df_openflight_routes = df_openflight_routes.drop(columns=['codeshare'])

print('\nResulting Data\n', df_openflight_routes)


Starting Data
       airline-code  airline-id source-airport-code  source-airport-id dest-airport-code  dest-airport-id codeshare  stops equipment
0               2B       410.0                 ASF             2966.0               KZN           2990.0       NaN      0       CR2
1               2B       410.0                 ASF             2966.0               MRV           2962.0       NaN      0       CR2
2               2B       410.0                 CEK             2968.0               KZN           2990.0       NaN      0       CR2
3               2B       410.0                 CEK             2968.0               OVB           4078.0       NaN      0       CR2
4               2B       410.0                 DME             4029.0               KZN           2990.0       NaN      0       CR2
...            ...         ...                 ...                ...               ...              ...       ...    ...       ...
67657           ZL      4178.0                 WYA          

In [204]:
# remove rows with missing data
# todo: can this be limited by excluding columns first?
no_missing_data = df_openflight_routes.notna().all(axis='columns')
print('\nData to be removed\n', df_openflight_routes[no_missing_data == False])

# df_openflight_routes = df_openflight_routes[no_missing_data]

# print('\nResulting Data\n', df_openflight_routes)


Data to be removed
       airline-code  airline-id source-airport-code  source-airport-id dest-airport-code  dest-airport-id  stops equipment
6               2B       410.0                 DME             4029.0               TGK              NaN      0       CR2
37              2B       410.0                 TGK                NaN               DME           4029.0      0       CR2
47              2G      1654.0                 IKT             2937.0               KCK              NaN      0       AN4
53              2G      1654.0                 KCK                NaN               IKT           2937.0      0       AN4
311             3H         NaN                 AKV             5506.0               YIK           5504.0      0       DH8
...            ...         ...                 ...                ...               ...              ...    ...       ...
67468           ZI        21.0                 MLH                NaN               CZL            221.0      0       320
674

In [205]:
# remove rows with invalid source airports
# todo: can this be limited by patching from other sources?
valid_source = df_openflight_routes['source-airport-id'].isin(df_openflight_airports['airport-id'])
print('\nData to be removed\n', df_openflight_routes[valid_source == False])
df_openflight_routes = df_openflight_routes[valid_source]
print('\nResulting Data\n', df_openflight_routes)


Data to be removed
       airline-code  airline-id source-airport-code  source-airport-id dest-airport-code  dest-airport-id  stops equipment
37              2B       410.0                 TGK                NaN               DME           4029.0      0       CR2
53              2G      1654.0                 KCK                NaN               IKT           2937.0      0       AN4
174             2O       146.0                 AOS             7167.0               KZB           7176.0      0       BNI
175             2O       146.0                 KKB             7168.0               SYB           7173.0      0       BNI
177             2O       146.0                 KOZ             7164.0               ORI           7165.0      0       BNI
...            ...         ...                 ...                ...               ...              ...    ...       ...
67470           ZI        21.0                 MLH                NaN               QSF           6492.0      0       320
675

In [206]:
# remove rows with invalid dest airports
# todo: can this be limited by patching from other sources?
valid_dest = df_openflight_routes['dest-airport-id'].isin(df_openflight_airports['airport-id'])
print('\nData to be removed\n', df_openflight_routes[valid_dest == False])
df_openflight_routes = df_openflight_routes[valid_dest]
print('\nResulting Data\n', df_openflight_routes)


Data to be removed
       airline-code  airline-id source-airport-code  source-airport-id dest-airport-code  dest-airport-id  stops equipment
6               2B       410.0                 DME             4029.0               TGK              NaN      0       CR2
47              2G      1654.0                 IKT             2937.0               KCK              NaN      0       AN4
169             2O       146.0                 ADQ             3531.0               AOS           7167.0      0       BNI
170             2O       146.0                 ADQ             3531.0               KKB           7168.0      0       BNI
172             2O       146.0                 ADQ             3531.0               KOZ           7164.0      0       BNI
...            ...         ...                 ...                ...               ...              ...    ...       ...
67501           ZI        21.0                 QSF             6492.0               MLH              NaN      0       320
675

In [209]:
# reorder and fix datatypes to prepare for graph population
df_openflight_routes = df_openflight_routes[['source-airport-id', 'dest-airport-id', 'airline-code', 'airline-id', 'source-airport-code', 'dest-airport-code', 'stops', 'equipment']]
df_openflight_routes['source-airport-id'] = df_openflight_routes['source-airport-id'].astype(str)
df_openflight_routes['dest-airport-id'] = df_openflight_routes['dest-airport-id'].astype(str)

print('\nResulting Data\n', df_openflight_routes)


Resulting Data
       source-airport-id dest-airport-id airline-code  airline-id source-airport-code dest-airport-code  stops equipment
0                2966.0          2990.0           2B       410.0                 ASF               KZN      0       CR2
1                2966.0          2962.0           2B       410.0                 ASF               MRV      0       CR2
2                2968.0          2990.0           2B       410.0                 CEK               KZN      0       CR2
3                2968.0          4078.0           2B       410.0                 CEK               OVB      0       CR2
4                4029.0          2990.0           2B       410.0                 DME               KZN      0       CR2
...                 ...             ...          ...         ...                 ...               ...    ...       ...
67657            6334.0          3341.0           ZL      4178.0                 WYA               ADL      0       SF3
67658            4029.0

In [210]:
# clean df_provided_routes

print(df_provided_routes)
# create dictionary for patching missing airport IDs from airport codes
iataLookup = df_openflight_airports.set_index('iata')['airport-id'].to_dict()

# patch source airport id where possible
missing_source_mask = df_provided_routes['source-airport-id'].isna()
df_provided_routes.loc[missing_source_mask, 'source-airport-id'] = df_provided_routes.loc[missing_source_mask, 'source-airport-code'].map(iataLookup)

# patch dest airport id where possible
missing_dest_mask = df_provided_routes['dest-airport-id'].isna()
df_provided_routes.loc[missing_dest_mask, 'dest-airport-id'] = df_provided_routes.loc[missing_dest_mask, 'dest-airport-code'].map(iataLookup)

# remove routes where airport IDs are still missing
# todo: can we just use airport codes everywhere instead?
missing_mask = df_provided_routes.isna().loc[:,('source-airport-id', 'dest-airport-id')].any(axis='columns')
df_provided_routes = df_provided_routes[missing_mask == False]

print(df_provided_routes)


      airline-code airline-id source-airport-code source-airport-id dest-airport-code dest-airport-id  stops equipment
0            AL242      AL242                 ASF              2966               KZN            2990      0       CR2
1            AL242      AL242                 ASF              2966               MRV            2962      0       CR2
2            AL242      AL242                 CEK              2968               KZN            2990      0       CR2
3            AL242      AL242                 CEK              2968               OVB            4078      0       CR2
4            AL242      AL242                 DME              4029               KZN            2990      0       CR2
...            ...        ...                 ...               ...               ...             ...    ...       ...
67657        AL156      AL156                 WYA              6334               ADL            3341      0       SF3
67658        AL530      AL530                 DM

In [211]:
# debugging: random airline lookup 
key = 'SQS'
print(df_provided_routes[df_provided_routes['source-airport-code'] == key])
print(df_provided_routes[df_provided_routes['dest-airport-code'] == key])
print(df_provided_airports[df_provided_airports['Code'] == key])
print(df_openflight_airports[df_openflight_airports['iata'] == key])
print(df_openflight_airports[df_openflight_airports['icao'] == key])

Empty DataFrame
Columns: [airline-code, airline-id, source-airport-code, source-airport-id, dest-airport-code, dest-airport-id, stops, equipment]
Index: []
Empty DataFrame
Columns: [airline-code, airline-id, source-airport-code, source-airport-id, dest-airport-code, dest-airport-id, stops, equipment]
Index: []
Empty DataFrame
Columns: [City, Country, Code]
Index: []
Empty DataFrame
Columns: [airport-id, name, city, country, iata, icao, latitude, longitude, altitude, timezone, DST, database-timezone, type, source]
Index: []
Empty DataFrame
Columns: [airport-id, name, city, country, iata, icao, latitude, longitude, altitude, timezone, DST, database-timezone, type, source]
Index: []


In [212]:
# compare openflight and provided data
# identify airline AL28

In [213]:
# create route graph from openflight
# https://igraph.org/python/api/latest/igraph.Graph.html#DataFrame
vertices = df_airports
directed = True
edges = df_routes
g = Graph.DataFrame(edges, directed, vertices)


# start=160
# size=1
# for i in range(start//size, len(vertices.index)//size-start):
#   print('rows',size*i, size*(i+1))
#   edges = df_routes[size*i:size*(i+1)]
#   try:
#     g = Graph.DataFrame(edges, directed, vertices)
#   except:
#     print(edges)
#     raise



NameError: ignored

In [ ]:
print(g.vcount(), g.ecount())

In [ ]:
# print(airportDict['3531'])
# print(df_airports[ df_airports['airport-id'] == '3531' ] )

# print(airportDict['7167'])
# print(df_airports[ df_airports['airport-id'] == '7167' ] )
# print(df_airports[ df_airports['airport-id'].str.contains('167') ] )
test = df_routes['source-airport-id'].isin(df_airports['airport-id'])
print(df_routes[ test==False ])

# print(df_airports[ df_airports['airport-id'] == '14106' ] )

In [ ]:
# load anonymized airline route data set (to be provided)
raw_column_names = ['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment']
df_anon_routes = pd.read_csv(
    'https://raw.githubusercontent.com/jpatokal/openflights/master/data/countries.dat',
    names=['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment'],
    skipinitialspace='true', header=0)

In [ ]:
# separate airline assigned to me

In [ ]:
# Part 1a
# Construct a graph (or a graph data structure) for the airline being analyzed by you. With
# airport code as the vertex, draw an undirected edge between the two airports in each
# line of the input. Since many Python packages/algorithms only accept integers as vertex,
# you may have to map the 3-letter airport code into an integer. You can use a dictionary
# or something else for this purpose.

In [ ]:
# Part 1b
# Using the given method, generate a few characteristics for this airline route graph. This
# will help understand some of the characteristics of the data set you are analyzing.
# Interpret them from an airlines operation perspective.

In [ ]:
# Part 1c
# Find the top 3 to 5 nodes (or airports; top k, in general) from where there are more
# number of flights than other nodes for that airline. These are termed hubs by an airline.
# You can use a node centrality detection algorithm/package for this

In [ ]:
# Part 1d
# Once you have the top k hubs, try to identify the airline that you are working on. You
# can use any data that is available on the Internet for this purpose. We will give you a file
# that maps airport code to airport names and cities.

In [ ]:
# Part 1e
# Once you have the airline identified, verify that with us. In this step of the analysis,
# your goal is to identify or predict the next hub for airline expansion. For this, you can
# use demographic and other information (e.g., regional) that is available on the web.
# The purpose of this step is for you to develop a feel for the analysis beyond data
# usage. We will grade you based on the ideas you use/come up with for this analysis. The
# actual airport identified as the next hub is secondary. If it matches with the actual one,
# that is great. You need to argue why your process and the publicly available data you
# have used are appropriate.